In [1]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
# Load chunks.txt and store it in a list, separated by newlines
with open('chunks.txt', 'r') as f:
    chunks = f.read().split('\n')


In [3]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
import os
import dotenv

dotenv.load_dotenv('env')
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']
PINECONE_API_ENV = os.environ['PINECONE_API_ENV']

In [5]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [6]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "langchain" # put in the name of your pinecone index here

In [8]:
docsearch = Pinecone.from_texts(chunks, embeddings, index_name=index_name)

In [9]:
query = "Reccomend me a game: I want to conquer castles. It should be singleplayer. It should have a large map. It should be set in the middle ages. It should be first person"
docs = docsearch.similarity_search(query, include_metadata=True)

In [10]:
# Example chunk
docs[0].page_content[:]

'Mount & Blade II: Bannerlord: as a young lad i served in the viking army, raided and looted innocent villages, sieged great castles and cities, and fought huge battles at the front with my powerful axe. i finally decided i was powerful enough to strike out on my own, i rebelled against my king, and was inmediately attacked by many thousands of troops, my cities and castles were destroyed, and me and my family were taken into slavery, and i lost all my millions of dollars. 10/10'

In [11]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [12]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [13]:
query = "Reccomend me a game: I want to conquer castles. It should be singleplayer. It should have a large map. It should be set in the middle ages. It should be first person"
docs = docsearch.similarity_search(query, include_metadata=True)

In [14]:
docs

[Document(page_content='Mount & Blade II: Bannerlord: as a young lad i served in the viking army, raided and looted innocent villages, sieged great castles and cities, and fought huge battles at the front with my powerful axe. i finally decided i was powerful enough to strike out on my own, i rebelled against my king, and was inmediately attacked by many thousands of troops, my cities and castles were destroyed, and me and my family were taken into slavery, and i lost all my millions of dollars. 10/10', metadata={}),
 Document(page_content='Europa Universalis IV: A rather complex grand strategy game, you pick a country to control starting from the year 1444 all the way to 1821. There isn\'t any fixed objective to aim for: it\'s  a good idea to try to get the achievements offered (such as world domination), as most are a real challenge to earn.   Each country is divided into provinces, allowing movement of you armies from one to another in real time - nothing here is turn based. Battles

In [15]:
chain.run(input_documents=docs, question=query)

' Mount & Blade II: Bannerlord is a great choice for you. It is a singleplayer game set in the middle ages where you can conquer castles. The game world is massive, with plenty of cities, fiefs, and castles to visit, plunder, or do trade with. You can also recruit troops and build up your army to help you in your conquests.'